<center>
<img src="../../img/ods_stickers.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #6 (demo)
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
**Same assignment as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a6-demo-linear-models-and-rf-for-regression) + [solution](https://www.kaggle.com/kashnitsky/a6-demo-regression-solution).**    
    
<img src='../../img/wine_quality.jpg' width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [2]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [14]:
y = data['quality']

feature_cols = [col for col in data.columns if col != 'quality']
X_data = data[feature_cols]

X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [31]:
linreg = LinearRegression(n_jobs=-1)
linreg.fit(X_train_scaled, y_train)

LinearRegression(n_jobs=-1)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [32]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, linreg.predict(X_test_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [46]:
linreg_coef = pd.DataFrame({'feature': feature_cols,
                            'coef': linreg.coef_})
linreg_coef['coef_importance'] = linreg_coef['coef'].apply(abs)
linreg_coef.sort_values(by='coef_importance', ascending=False)

,feature,coef,coef_importance
7,density,-0.665720,0.665720
3,residual sugar,0.538164,0.538164
1,volatile acidity,-0.192260,0.192260
8,pH,0.150036,0.150036
10,alcohol,0.129533,0.129533
0,fixed acidity,0.097822,0.097822
9,sulphates,0.062053,0.062053
5,free sulfur dioxide,0.042180,0.042180
6,total sulfur dioxide,0.014304,0.014304
4,chlorides,0.008127,0.008127


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [35]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [47]:
lasso1_coef = pd.DataFrame({'feature': feature_cols,
                            'coef': lasso1.coef_})
lasso1_coef['coef_importance'] = lasso1_coef['coef'].apply(abs)
lasso1_coef.sort_values(by='coef_importance', ascending=False)

,feature,coef,coef_importance
10,alcohol,0.322425,0.322425
3,residual sugar,0.256363,0.256363
7,density,-0.235492,0.235492
1,volatile acidity,-0.188479,0.188479
8,pH,0.067277,0.067277
5,free sulfur dioxide,0.043088,0.043088
9,sulphates,0.029722,0.029722
4,chlorides,-0.002747,0.002747
0,fixed acidity,-0.000000,0.000000
2,citric acid,-0.000000,0.000000


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [43]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5, alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [44]:
lasso_cv.alpha_

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [49]:
lasso_cv_coef = pd.DataFrame({'feature': feature_cols,
                              'coef': lasso_cv.coef_})
lasso_cv_coef['coef_importance'] = lasso_cv_coef['coef'].apply(abs)
lasso_cv_coef.sort_values(by='coef_importance')

,feature,coef,coef_importance
2,citric acid,-0.000000,0.000000
4,chlorides,0.006933,0.006933
6,total sulfur dioxide,0.012969,0.012969
5,free sulfur dioxide,0.042698,0.042698
9,sulphates,0.060939,0.060939
0,fixed acidity,0.093295,0.093295
10,alcohol,0.137115,0.137115
8,pH,0.146549,0.146549
1,volatile acidity,-0.192049,0.192049
3,residual sugar,0.526883,0.526883


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [50]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, lasso_cv.predict(X_test_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [51]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [63]:
def mean_cv_mse(estimator, X, y):
    cv_neg_scores = cross_val_score(estimator, X, y, scoring='neg_mean_squared_error')
    return abs(cv_neg_scores.mean())

print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % mean_cv_mse(forest, X_train_scaled, y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, forest.predict(X_test_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [67]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, scoring='neg_mean_squared_error',
                                   n_jobs=-1, verbose=1)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 3150 out of 3150 | elapsed: 11.9min finished


GridSearchCV(estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
             scoring='neg_mean_squared_error', verbose=1)

In [68]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 -0.39773288191505934)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [72]:
print("Mean squared error (cv): %.3f" % mean_cv_mse(locally_best_forest.best_estimator_, X_train_scaled, y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, locally_best_forest.predict(X_test_scaled)))

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [80]:
rf_importance = pd.DataFrame({'feature': feature_cols,
                              'coef': locally_best_forest.best_estimator_.feature_importances_})
rf_importance['coef_importance'] = rf_importance['coef'].apply(abs)
rf_importance.sort_values(by='coef_importance', ascending=False)

,feature,coef,coef_importance
10,alcohol,0.206056,0.206056
1,volatile acidity,0.117578,0.117578
5,free sulfur dioxide,0.111556,0.111556
7,density,0.088549,0.088549
8,pH,0.073659,0.073659
6,total sulfur dioxide,0.073640,0.073640
4,chlorides,0.073366,0.073366
3,residual sugar,0.072072,0.072072
2,citric acid,0.062601,0.062601
0,fixed acidity,0.061813,0.061813


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**

Random Forest model has higher score than both Linear and Lasso models. Difference between RF and Lin + Lasso is in fact that the first one is not linear => wine quality doesn't depend from wine features in linear way.

Joke: RF model showed that alcohol feature is more important